In [7]:

import requests
import gzip
import json
import traceback


from pyspark.sql import SQLContext
from pyspark.sql import functions as F  
from pyspark.sql import types as T

In [15]:
# events_list is a list of dicts to store all the json files from GitHub-Archive
events_list = []


def extract_data():
  # extracts json.gz from GHArchive and loads events_list with the data
  
  for hour in range(12):
    url = 'http://data.gharchive.org/2015-01-01-'+str(hour)+'.json.gz'
    r = requests.get(url)
    file_path = "../datasets/"+url.split("/")[-1]

    with open(file_path, 'wb') as f:
        f.write(r.content)

    with gzip.open(file_path, 'rb') as f:
        for line in f:
            event = json.loads(line.decode("utf-8"))
            events_list.append(event)
  return

In [16]:
    extract_data()


In [19]:

# To create dataframe directly from events_list, we need to create schema explicitly.
# The events_list is nested with more than 7 layers deep and hundreds of fields.
# Instead dumping it as a json in DBFS (DataBricks File System) and creating 
# dataframe from json automatically infers the schema. 

# dumping archive extract into DBFS
file_path = "../datasets/2015-01-01.json"   
with open(file_path, 'w+') as fout:
  json.dump(events_list, fout)

# creating dataframe from json files 
file_path = "../datasets/2015-01-01.json"
master_df = spark.read.format("json").load(file_path)

NameError: name 'spark' is not defined

In [ ]:

main_df = master_df.select(F.col("actor.id").alias("actor_id"), 
                           F.col("org.id").alias("org_id"), 
                           F.col("id").alias("event_id"), 
                           F.col("created_at"), 
                           F.col("repo.id").alias("repo_id"), 
                           F.col("type").alias("event_type"))
# main_df.show()